In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import lightgbm as lgb

In [ ]:
import pandas as pd
import os
data_path = '/Users/favea/Downloads/swiss-data'

weather_files = [os.path.join(data_path,f) for f in os.listdir(data_path) if 'weather' in f]

cleaned_df = pd.read_csv(os.path.join(data_path,'10122022b_cleaned.csv'),index_col=0)

In [ ]:
df_weather = None
for wf in weather_files:
    print(wf)
    if df_weather is None:
        df_weather = pd.read_csv(wf)
        print(df_weather.shape)
    else:
        d = pd.read_csv(wf)
        df_weather = df_weather.append(d)



In [ ]:
merged = pd.merge(left = cleaned_df, right = df_weather, on = ['origin','destination','scheduled_time_departure'])

In [ ]:
merged = merged.dropna().drop(['year','delay_minutes','scheduled_time_departure','Unnamed: 0_x','Unnamed: 0_y'],axis=1)

without weather feature

In [ ]:
weather_features = ['temperature','precipitation', 'windSpeed', 'dewpoint', 'humidity']

In [ ]:
X,y = merged.drop('delayed',1), merged.loc[:,'delayed']

In [ ]:
X = X.drop(weather_features,axis=1)
X.carrier = X.carrier.astype('category')
X.origin = X.origin.astype('category')
X.destination = X.destination.astype('category')
X.previous_is_delayed = X.previous_is_delayed.astype('category')

In [ ]:
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed, stratify=y)


model = LGBMClassifier()
gbm = model.fit(X_train, y_train, eval_set=(X_test,y_test), eval_metric ='average_precision')




In [ ]:
lgb.plot_importance(gbm)

In [ ]:
lgb.plot_metric(gbm,'average_precision')

In [ ]:
y_test_pred = gbm.predict(X_test, num_iteration = gbm.best_iteration_)
from sklearn.metrics import classification_report
print(classification_report(y_test, (y_test_pred>=0.5).astype(int)))

with weather feature

In [ ]:
X,y = merged.drop('delayed',1), merged.loc[:,'delayed']

In [ ]:
X.carrier = X.carrier.astype('category')
X.origin = X.origin.astype('category')
X.destination = X.destination.astype('category')
X.previous_is_delayed = X.previous_is_delayed.astype('category')

In [ ]:
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed, stratify=y)

In [ ]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
model = LGBMClassifier()
gbm = model.fit(X_train, y_train, eval_set=(X_test,y_test), eval_metric ='average_precision')


In [ ]:
import lightgbm as lgb

In [ ]:
lgb.plot_importance(gbm)

In [ ]:
lgb.plot_metric(gbm,'average_precision')

In [ ]:
y_test_pred = gbm.predict(X_test, num_iteration = gbm.best_iteration_)
from sklearn.metrics import classification_report
print(classification_report(y_test, (y_test_pred>=0.5).astype(int)))